# Introduction

This project is to analyse the healthcare facilities in Nairobi kenya and give recommendations addressing healthcare accessibility, demographic disparities, and healthcare system improvements.

# Import Modules

In [85]:
import pandas as pd
import pathlib
import geopandas as gpd
import plotly.express as px

Resolve paths

In [86]:
mainpath = pathlib.Path()

#resolve path
mainpath = mainpath.resolve()

#specify folders
inpath = mainpath/'data'
outpath = mainpath/'output'

In [87]:
#specify files to use

health_facilities = inpath/'ehealth-kenya-facilities-download-21102015.xls'
county_population = inpath/'kenya-population-by-sex-and-county.csv'
sub_county_population = inpath/'kenya-population-by-sub-county.csv'
sub_county_density = inpath/'kenya-populationland-area-population-density_by_subcounty.csv'
sub_county_shp = inpath/'Sub counties shapefile/Nairobi Sub counties.shp'
county_shp = inpath/'Sub counties shapefile/Nairobi County.shp'

# Read Files

In [88]:
# read in files into pandas dataframes
health = pd.read_excel(health_facilities, 
                           usecols=['Facility Code', 'Facility Name', 'County', 'Type', 'Owner', 
                                    'Open 24 Hours', 'Open Weekends', 'Operational Status', 'Constituency'])
county_pop = pd.read_csv(county_population, skiprows=6)
sub_pop = pd.read_csv(sub_county_population, skiprows=6, usecols=range(0,5), )
sub_pop_dens = pd.read_csv(sub_county_density, usecols=['National/ County', 'Land Area (Sq. Km)', 'Population Density (No. per Sq. Km)'], skiprows=5)
sub_gdf = gpd.read_file(sub_county_shp)
county_gdf = gpd.read_file(county_shp)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


# Data Cleaning & Preprocessing

We will select operational hospitals only in Nairobi county

In [89]:
#hospitals in Nairobi
health = health[health['County'] == 'Nairobi']

#select operational hospitals only
health = health[health['Operational Status'] == "Operational"]

health

,Facility Code,Facility Name,County,Type,Owner,Constituency,Open 24 Hours,Open Weekends,Operational Status
1,19310,St Jude's Huruma Community Health Services,Nairobi,Medical Clinic,Private Practice - Unspecified,MATHARE,N,Y,Operational
7,13043,7Kr Mrs Health Centre,Nairobi,Health Centre,Armed Forces,LANG'ATA,Y,Y,Operational
10,20346,AAR Adams Health Centre,Nairobi,Medical Clinic,Private Practice - General Practitioner,KIBRA,N,Y,Operational
11,12861,AAR City Centre Clinic,Nairobi,Medical Clinic,Private Enterprise (Institution),STAREHE,N,Y,Operational
12,16796,AAR Clinic Sarit Centre (Westlands),Nairobi,Medical Clinic,Private Enterprise (Institution),WESTLANDS,N,N,Operational
...,...,...,...,...,...,...,...,...,...
10459,13260,Woodstreet Nursing Home,Nairobi,Nursing Home,Private Enterprise (Institution),KAMUKUNJI,N,N,Operational
10493,19273,Zimerbreeze Medical Centre,Nairobi,Medical Clinic,Private Practice - General Practitioner,ROYSAMBU,N,Y,Operational
10494,19378,Zimma Health Care,Nairobi,Medical Clinic,Private Practice - General Practitioner,ROYSAMBU,N,Y,Operational
10495,13261,Zimmerman Medical Dispensary,Nairobi,Dispensary,Private Enterprise (Institution),ROYSAMBU,N,N,Operational


## Cleaning County Data

In [90]:
county_pop = county_pop.tail(1)
county_pop

,name,Male,Female,Intersex,Total
47,Nairobi,2192452,2204376,245,4397073


## Cleaning the Sub County population Data

The nairobi Sub counties are the last 12 rows

In [91]:
#select the last 12 rows
sub_pop = sub_pop.tail(11)

#reset index
sub_pop.reset_index(drop=True, inplace=True)

#remove whitespace
sub_pop['name'] = sub_pop['name'].str.strip()

sub_pop

,name,Male,Female,Intersex,Total
0,Dagoretti,217651,216526,31.0,434208
1,Embakasi,492476,496270,62.0,988808
2,Kamukunji,136670,131599,7.0,268276
3,Kasarani,381234,399385,37.0,780656
4,Kibra,94199,91569,9.0,185777
5,Lang'ata,96698,100774,17.0,197489
6,Makadara,96369,93157,10.0,189536
7,Mathare,106522,100028,14.0,206564
8,Njiru,307642,318809,31.0,626482
9,Starehe,109173,101238,12.0,210423


In [92]:
#select the last 12 rows
sub_pop_dens = sub_pop_dens.tail(11)

#reset index
sub_pop_dens.reset_index(drop=True, inplace=True)

#remove whitespace
sub_pop_dens['National/ County'] = sub_pop_dens['National/ County'].str.strip()

sub_pop_dens

,National/ County,Land Area (Sq. Km),Population Density (No. per Sq. Km)
0,Dagoretti,29.1,14908
1,Embakasi,86.3,11460
2,Kamukunji,10.5,25455
3,Kasarani,86.2,9058
4,Kibra,12.1,15311
5,Lang'ata,216.8,911
6,Makadara,11.7,16150
7,Mathare,3.0,68941
8,Njiru,129.9,4821
9,Starehe,20.6,10205


## Merge population Data

let's merge population density and population

In [93]:
#merge
sub_data = pd.merge(sub_pop, sub_pop_dens, left_on='name', right_on='National/ County')

#drop columns
sub_data.drop(columns=('National/ County'), inplace=True)

sub_data

,name,Male,Female,Intersex,Total,Land Area (Sq. Km),Population Density (No. per Sq. Km)
0,Dagoretti,217651,216526,31.0,434208,29.1,14908
1,Embakasi,492476,496270,62.0,988808,86.3,11460
2,Kamukunji,136670,131599,7.0,268276,10.5,25455
3,Kasarani,381234,399385,37.0,780656,86.2,9058
4,Kibra,94199,91569,9.0,185777,12.1,15311
5,Lang'ata,96698,100774,17.0,197489,216.8,911
6,Makadara,96369,93157,10.0,189536,11.7,16150
7,Mathare,106522,100028,14.0,206564,3.0,68941
8,Njiru,307642,318809,31.0,626482,129.9,4821
9,Starehe,109173,101238,12.0,210423,20.6,10205


The population data is limited to only a few sub counties,  we wil merge the hosptial data i.e dagoretti north and south as well as Embakasi and merge the available datasets 

We want to check the number of hospitals in each sub county and to find the health facilities citizens have access to. To do this we will group by sub_county and aggregate the number of health facilities.

# County Level Analysis

Now that we have cleaned the data, lets see how many operational hospitals are in Nairobi county

In [94]:
health.reset_index(drop=True, inplace=True)
health

,Facility Code,Facility Name,County,Type,Owner,Constituency,Open 24 Hours,Open Weekends,Operational Status
0,19310,St Jude's Huruma Community Health Services,Nairobi,Medical Clinic,Private Practice - Unspecified,MATHARE,N,Y,Operational
1,13043,7Kr Mrs Health Centre,Nairobi,Health Centre,Armed Forces,LANG'ATA,Y,Y,Operational
2,20346,AAR Adams Health Centre,Nairobi,Medical Clinic,Private Practice - General Practitioner,KIBRA,N,Y,Operational
3,12861,AAR City Centre Clinic,Nairobi,Medical Clinic,Private Enterprise (Institution),STAREHE,N,Y,Operational
4,16796,AAR Clinic Sarit Centre (Westlands),Nairobi,Medical Clinic,Private Enterprise (Institution),WESTLANDS,N,N,Operational
...,...,...,...,...,...,...,...,...,...
912,13260,Woodstreet Nursing Home,Nairobi,Nursing Home,Private Enterprise (Institution),KAMUKUNJI,N,N,Operational
913,19273,Zimerbreeze Medical Centre,Nairobi,Medical Clinic,Private Practice - General Practitioner,ROYSAMBU,N,Y,Operational
914,19378,Zimma Health Care,Nairobi,Medical Clinic,Private Practice - General Practitioner,ROYSAMBU,N,Y,Operational
915,13261,Zimmerman Medical Dispensary,Nairobi,Dispensary,Private Enterprise (Institution),ROYSAMBU,N,N,Operational


## Number of the different types of hospitals in the County

In [95]:
#lets check the number of the different types of hospitals
health_types = health.groupby(['Type']).agg(Type_count=('Type', 'count')).reset_index()
health_types.head()

,Type,Type_count
0,Dental Clinic,10
1,Dispensary,190
2,District Health Office,1
3,District Hospital,3
4,Eye Centre,2


In [96]:
#plot
fig = px.pie(health_types, names='Type', values='Type_count', width=1280, height=720, title="Types of Health Facilities in Nairobi County")
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

Which are the the two referral hospitals

In [97]:
national_ref = health[health['Type'] == 'National Referral Hospital']
national_ref

,Facility Code,Facility Name,County,Type,Owner,Constituency,Open 24 Hours,Open Weekends,Operational Status
186,13023,Kenyatta National Hospital,Nairobi,National Referral Hospital,Ministry of Health,KIBRA,Y,Y,Operational
615,13194,National Spinal Injury Hospital,Nairobi,National Referral Hospital,Ministry of Health,DAGORETTI NORTH,Y,Y,Operational


## Health Facility Ownership

Let's see who mostly owns the health facilities

In [98]:
health_owners = health.groupby(['Owner']).agg(Owner_count=('Owner', 'count')).reset_index()
health_owners.head()

,Owner,Owner_count
0,Academic (if registered),6
1,Armed Forces,9
2,Christian Health Association of Kenya,21
3,Community,11
4,Company Medical Service,3


In [99]:
#plot
fig = px.pie(health_owners, names='Owner', values='Owner_count', width=1280, height=720, title="Health Facilities Ownership in Nairobi County")
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

## Health Facility Ownership and Type of Hospital

Let's check the who are the owners of the different categries of hospitals

In [100]:
health_owner_type = health.groupby(['Type', 'Owner']).agg(Count=('Type', 'count')).reset_index()
health_owner_type.head()

,Type,Owner,Count
0,Dental Clinic,Local Authority,1
1,Dental Clinic,Private Practice - Unspecified,9
2,Dispensary,Academic (if registered),3
3,Dispensary,Armed Forces,5
4,Dispensary,Christian Health Association of Kenya,9


In [101]:
#plot
fig = px.bar(health_owner_type, 'Owner','Count', color='Type', title='Health Facilties and their Management' , text='Type' ,height=720, width=1280)
fig.show()

# Sub County Level Analysis

## Hospitals as per Sub Counties
Let's see how many hospitals are in each sub county. The constituencies column represents the current sub-counties

In [102]:
health_sub = health.groupby(['Constituency']).agg(Count=('Type', 'count')).reset_index()
health_sub

,Constituency,Count
0,DAGORETTI NORTH,76
1,DAGORETTI SOUTH,29
2,EMBAKASI CENTRAL,42
3,EMBAKASI EAST,27
4,EMBAKASI NORTH,12
5,EMBAKASI SOUTH,26
6,EMBAKASI WEST,38
7,KAMUKUNJI,59
8,KASARANI,67
9,KIBRA,81


In [103]:
#plot
fig = px.pie(health_sub, 
             names='Constituency', 
             values='Count', 
             #title='Number of Health Facilities in Each Sub County', 
             height=720, width=1280)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

## Number of the Different Types of Health Facilities in the Constituencies
Let's check and see the different types of health facilities in the county

In [104]:
health_sub_type = health.groupby(['Constituency', 'Type']).agg(Count=('Type', 'count')).reset_index()
health_sub_type.head()

,Constituency,Type,Count
0,DAGORETTI NORTH,Dental Clinic,1
1,DAGORETTI NORTH,Dispensary,21
2,DAGORETTI NORTH,Health Centre,13
3,DAGORETTI NORTH,Health Programme,1
4,DAGORETTI NORTH,Medical Centre,1


In [105]:
#let's plot the data
fig = px.bar(
    health_sub_type, 'Constituency', 'Count', title='Number of Different Types of Health Facilities in each Sub County', 
    width=1280, height=720, color='Type', text='Type'
)
fig.show()

## Government Owned Hospitals
Which hospitals are government owned 

In [106]:
govt = ['Ministry of Health', 'Other Public Institution', 'Parastatal', 'State Coorporation', 'Armed Forces', 'Local Authority']
health_govt = health[health['Owner'].isin(govt)]
health_govt

,Facility Code,Facility Name,County,Type,Owner,Constituency,Open 24 Hours,Open Weekends,Operational Status
1,13043,7Kr Mrs Health Centre,Nairobi,Health Centre,Armed Forces,LANG'ATA,Y,Y,Operational
33,13231,State House Clinic,Nairobi,Dispensary,Local Authority,DAGORETTI NORTH,N,N,Operational
34,13232,State House Dispensary (Nairobi),Nairobi,Dispensary,Ministry of Health,DAGORETTI NORTH,N,N,Operational
38,18304,The Co-Operative University College of Kenya D...,Nairobi,Dispensary,Parastatal,LANG'ATA,N,N,Operational
50,18344,Afya House Dispensary,Nairobi,Dispensary,Ministry of Health,KIBRA,N,N,Operational
...,...,...,...,...,...,...,...,...,...
895,13249,Waithaka Health Centre,Nairobi,Health Centre,Local Authority,DAGORETTI SOUTH,Y,Y,Operational
901,18860,Wellness Program KWS Hq,Nairobi,Medical Clinic,Parastatal,LANG'ATA,N,N,Operational
907,19663,Westlands District Health Management Team,Nairobi,Medical Clinic,Ministry of Health,WESTLANDS,Y,N,Operational
908,13258,Westlands Health Centre,Nairobi,Health Centre,Local Authority,WESTLANDS,Y,Y,Operational


Let's see how they are distributed across the constituncies

In [107]:
health_sub_govt = health_govt.groupby(['Constituency', 'Owner', 'Type']).agg(Count=('Type', 'count')).reset_index()
health_sub_govt.head()

,Constituency,Owner,Type,Count
0,DAGORETTI NORTH,Armed Forces,Dispensary,1
1,DAGORETTI NORTH,Local Authority,Dental Clinic,1
2,DAGORETTI NORTH,Local Authority,Dispensary,2
3,DAGORETTI NORTH,Ministry of Health,Dispensary,1
4,DAGORETTI NORTH,Ministry of Health,National Referral Hospital,1


Let's plot the results

In [108]:
fig = px.bar(
    health_sub_govt, x="Owner", y="Count", color="Type",
             facet_col="Constituency",
             barmode="overlay",
             facet_col_wrap=3,
             height=1080,
             title = 'Govt. Owned Health Facility Types in Each Sub County '
)


fig.show()

Let's plot the Number of government backed  Hospitals in each Sub County

In [109]:
health_sub_govt = health_govt.groupby(['Constituency', 'Owner']).agg(Count=('Type', 'count')).reset_index()

fig = px.bar(
    health_sub_govt, 'Constituency', 'Count', title='Number of Govt. Health Facilities in Each Sub County', 
    width=1280, height=720, color='Owner', text='Owner'
)
fig.show()

# Contituency Hospitals and Population
We will look at the hospital data with repect to the population, its worth noting that the subcounties which were the 2019 census used have changed, we will just use a few of them which corelate.To do this we will merge some of the hospital data.

## Clean and Merge the Data

In [110]:
#copy the df
health_sub2 = health_sub.copy()

#relable the values
health_sub2['Constituency'] = health_sub2['Constituency'].replace({
        'DAGORETTI NORTH':'DAGORETTI', 
        'DAGORETTI SOUTH':'DAGORETTI',
        'EMBAKASI CENTRAL':'EMBAKASI',
        'EMBAKASI EAST':'EMBAKASI',
        'EMBAKASI NORTH':'EMBAKASI',
        'EMBAKASI SOUTH':'EMBAKASI',
        'EMBAKASI WEST':'EMBAKASI'
    }
)

#merge and sum
health_sub2 = health_sub2.groupby(['Constituency']).agg(Health_Facilities=('Count', 'sum')).reset_index()

health_sub2

,Constituency,Health_Facilities
0,DAGORETTI,105
1,EMBAKASI,145
2,KAMUKUNJI,59
3,KASARANI,67
4,KIBRA,81
5,LANG'ATA,68
6,MAKADARA,51
7,MATHARE,16
8,ROYSAMBU,79
9,RUARAKA,38


In [111]:
# convert to uppercase
sub_data['name'] = sub_data['name'].str.upper()
sub_data

,name,Male,Female,Intersex,Total,Land Area (Sq. Km),Population Density (No. per Sq. Km)
0,DAGORETTI,217651,216526,31.0,434208,29.1,14908
1,EMBAKASI,492476,496270,62.0,988808,86.3,11460
2,KAMUKUNJI,136670,131599,7.0,268276,10.5,25455
3,KASARANI,381234,399385,37.0,780656,86.2,9058
4,KIBRA,94199,91569,9.0,185777,12.1,15311
5,LANG'ATA,96698,100774,17.0,197489,216.8,911
6,MAKADARA,96369,93157,10.0,189536,11.7,16150
7,MATHARE,106522,100028,14.0,206564,3.0,68941
8,NJIRU,307642,318809,31.0,626482,129.9,4821
9,STAREHE,109173,101238,12.0,210423,20.6,10205


In [112]:
#merge the dataframes
health_sub_pop = pd.merge(sub_data, health_sub2, left_on='name', right_on='Constituency')

#drop columns
health_sub_pop.drop(columns=['Constituency'], inplace=True)

#sort values
health_sub_pop = health_sub_pop.sort_values('Total' ,ascending=False)

health_sub_pop

,name,Male,Female,Intersex,Total,Land Area (Sq. Km),Population Density (No. per Sq. Km),Health_Facilities
1,EMBAKASI,492476,496270,62.0,988808,86.3,11460,145
3,KASARANI,381234,399385,37.0,780656,86.2,9058,67
0,DAGORETTI,217651,216526,31.0,434208,29.1,14908,105
9,WESTLANDS,153818,155021,15.0,308854,97.5,3167,71
2,KAMUKUNJI,136670,131599,7.0,268276,10.5,25455,59
8,STAREHE,109173,101238,12.0,210423,20.6,10205,137
7,MATHARE,106522,100028,14.0,206564,3.0,68941,16
5,LANG'ATA,96698,100774,17.0,197489,216.8,911,68
6,MAKADARA,96369,93157,10.0,189536,11.7,16150,51
4,KIBRA,94199,91569,9.0,185777,12.1,15311,81


## Sorting by Population and Population Density
Which area has the highest population and population density vis a vis Health Facilities

In [113]:
#plot
fig = px.scatter(
    health_sub_pop, 'Health_Facilities', 'Total', size='Population Density (No. per Sq. Km)', color='name', size_max=50, 
    text='name', height=720, width=1280, title= 'Population vs Number of Health Facilities'
)

fig.update_layout(
    xaxis_title='Number of Heath Facilities',
    yaxis_title='Total Population',
    legend_title='Sub County / Constituency',
)

fig.show()

# Geocoding Hospital Data

Inorder to get better analysis of the data on the sub-county level, we will geocode the hospital data to find inorder to merge with the population data down the line

In [114]:
'''from geopandas.tools import geocode

key = ''

health_geo = geocode(health['Facility Name'], provider='google', api_key=key)

health_geo.head()'''

"from geopandas.tools import geocode\n\nkey = ''\n\nhealth_geo = geocode(health['Facility Name'], provider='google', api_key=key)\n\nhealth_geo.head()"

In [115]:
#exporting geocoded data
'''health_geo.to_csv(outpath/'health_geo.csv')'''

"health_geo.to_csv(outpath/'health_geo.csv')"

## Merging Datasets

We will import our geocoded hospital data and merge with health data

In [116]:
health_geo = pd.read_csv(outpath/'health_geoc.csv')
health_geo

,Easting (X),Northing (Y),Hosp
0,36.872091,-1.256511,"Airbase, Nairobi, Kenya"
1,36.747626,-1.364094,"Nairobi, Kenya"
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,36.801300,-1.261629,"Sarit Center, 4th Floor, Nairobi, Kenya"
...,...,...,...
912,36.860009,-1.273179,"Nairobi, Kenya"
913,NaN,NaN,NaN
914,36.886906,-1.218459,"Zimmerman, Nairobi, Kenya"
915,NaN,NaN,NaN


In [117]:
#merge with health facilities data
health_geo = pd.merge(health, health_geo, left_index=True, right_index=True)
health_geo

,Facility Code,Facility Name,County,Type,Owner,Constituency,Open 24 Hours,Open Weekends,Operational Status,Easting (X),Northing (Y),Hosp
0,19310,St Jude's Huruma Community Health Services,Nairobi,Medical Clinic,Private Practice - Unspecified,MATHARE,N,Y,Operational,36.872091,-1.256511,"Airbase, Nairobi, Kenya"
1,13043,7Kr Mrs Health Centre,Nairobi,Health Centre,Armed Forces,LANG'ATA,Y,Y,Operational,36.747626,-1.364094,"Nairobi, Kenya"
2,20346,AAR Adams Health Centre,Nairobi,Medical Clinic,Private Practice - General Practitioner,KIBRA,N,Y,Operational,NaN,NaN,NaN
3,12861,AAR City Centre Clinic,Nairobi,Medical Clinic,Private Enterprise (Institution),STAREHE,N,Y,Operational,NaN,NaN,NaN
4,16796,AAR Clinic Sarit Centre (Westlands),Nairobi,Medical Clinic,Private Enterprise (Institution),WESTLANDS,N,N,Operational,36.801300,-1.261629,"Sarit Center, 4th Floor, Nairobi, Kenya"
...,...,...,...,...,...,...,...,...,...,...,...,...
912,13260,Woodstreet Nursing Home,Nairobi,Nursing Home,Private Enterprise (Institution),KAMUKUNJI,N,N,Operational,36.860009,-1.273179,"Nairobi, Kenya"
913,19273,Zimerbreeze Medical Centre,Nairobi,Medical Clinic,Private Practice - General Practitioner,ROYSAMBU,N,Y,Operational,NaN,NaN,NaN
914,19378,Zimma Health Care,Nairobi,Medical Clinic,Private Practice - General Practitioner,ROYSAMBU,N,Y,Operational,36.886906,-1.218459,"Zimmerman, Nairobi, Kenya"
915,13261,Zimmerman Medical Dispensary,Nairobi,Dispensary,Private Enterprise (Institution),ROYSAMBU,N,N,Operational,NaN,NaN,NaN


In [118]:
#convert to geodataframe
health_gdf = gpd.GeoDataFrame(health_geo, geometry=gpd.points_from_xy(health_geo['Easting (X)'], health_geo['Northing (Y)']), crs='EPSG:4326')

#drop columns
health_gdf.drop(columns=['Hosp'], inplace= True)

#drop na_values
health_gdf =health_gdf.dropna(subset=['Easting (X)'])

#reset index
health_gdf.reset_index(drop=True, inplace=True)

health_gdf

,Facility Code,Facility Name,County,Type,Owner,Constituency,Open 24 Hours,Open Weekends,Operational Status,Easting (X),Northing (Y),geometry
0,19310,St Jude's Huruma Community Health Services,Nairobi,Medical Clinic,Private Practice - Unspecified,MATHARE,N,Y,Operational,36.872091,-1.256511,POINT (36.87209 -1.25651)
1,13043,7Kr Mrs Health Centre,Nairobi,Health Centre,Armed Forces,LANG'ATA,Y,Y,Operational,36.747626,-1.364094,POINT (36.74763 -1.36409)
2,16796,AAR Clinic Sarit Centre (Westlands),Nairobi,Medical Clinic,Private Enterprise (Institution),WESTLANDS,N,N,Operational,36.801300,-1.261629,POINT (36.80130 -1.26163)
3,20140,AAR Mountain mall,Nairobi,Medical Clinic,Private Practice - Nurse / Midwife,ROYSAMBU,N,Y,Operational,36.873781,-1.232811,POINT (36.87378 -1.23281)
4,21224,AAR Outpatient Center Donholm,Nairobi,Medical Clinic,Private Practice - General Practitioner,EMBAKASI EAST,N,Y,Operational,36.821126,-1.284289,POINT (36.82113 -1.28429)
...,...,...,...,...,...,...,...,...,...,...,...,...
605,19494,Westlands Medical Centre,Nairobi,Medical Clinic,Non-Governmental Organizations,WESTLANDS,Y,Y,Operational,36.803648,-1.264548,POINT (36.80365 -1.26455)
606,13259,Woodley Clinic,Nairobi,Dispensary,Local Authority,KIBRA,N,N,Operational,36.787978,-1.311423,POINT (36.78798 -1.31142)
607,13260,Woodstreet Nursing Home,Nairobi,Nursing Home,Private Enterprise (Institution),KAMUKUNJI,N,N,Operational,36.860009,-1.273179,POINT (36.86001 -1.27318)
608,19378,Zimma Health Care,Nairobi,Medical Clinic,Private Practice - General Practitioner,ROYSAMBU,N,Y,Operational,36.886906,-1.218459,POINT (36.88691 -1.21846)


## Selecting Geocoded Hospitals Falling within Nairobi County

In [119]:
#lets mask the hospitasls geocoded within Nairobi
health_mask = health_gdf.within(county_gdf.at[0, 'geometry'])

health_gdf = health_gdf.loc[health_mask]

health_gdf

,Facility Code,Facility Name,County,Type,Owner,Constituency,Open 24 Hours,Open Weekends,Operational Status,Easting (X),Northing (Y),geometry
0,19310,St Jude's Huruma Community Health Services,Nairobi,Medical Clinic,Private Practice - Unspecified,MATHARE,N,Y,Operational,36.872091,-1.256511,POINT (36.87209 -1.25651)
1,13043,7Kr Mrs Health Centre,Nairobi,Health Centre,Armed Forces,LANG'ATA,Y,Y,Operational,36.747626,-1.364094,POINT (36.74763 -1.36409)
2,16796,AAR Clinic Sarit Centre (Westlands),Nairobi,Medical Clinic,Private Enterprise (Institution),WESTLANDS,N,N,Operational,36.801300,-1.261629,POINT (36.80130 -1.26163)
3,20140,AAR Mountain mall,Nairobi,Medical Clinic,Private Practice - Nurse / Midwife,ROYSAMBU,N,Y,Operational,36.873781,-1.232811,POINT (36.87378 -1.23281)
4,21224,AAR Outpatient Center Donholm,Nairobi,Medical Clinic,Private Practice - General Practitioner,EMBAKASI EAST,N,Y,Operational,36.821126,-1.284289,POINT (36.82113 -1.28429)
...,...,...,...,...,...,...,...,...,...,...,...,...
605,19494,Westlands Medical Centre,Nairobi,Medical Clinic,Non-Governmental Organizations,WESTLANDS,Y,Y,Operational,36.803648,-1.264548,POINT (36.80365 -1.26455)
606,13259,Woodley Clinic,Nairobi,Dispensary,Local Authority,KIBRA,N,N,Operational,36.787978,-1.311423,POINT (36.78798 -1.31142)
607,13260,Woodstreet Nursing Home,Nairobi,Nursing Home,Private Enterprise (Institution),KAMUKUNJI,N,N,Operational,36.860009,-1.273179,POINT (36.86001 -1.27318)
608,19378,Zimma Health Care,Nairobi,Medical Clinic,Private Practice - General Practitioner,ROYSAMBU,N,Y,Operational,36.886906,-1.218459,POINT (36.88691 -1.21846)


## Merging Sub County Geodataframe and Population

In [120]:
#clean the geodataframe 
sub_gdf.drop(columns=['GEO_CONCAT', 'GEO_MATCH', 'ADM1_NAME', 'ADM2_NAME', 'CNTRY_NAME' ,'ADM_LEVEL', 'USCBCMNT', 'Shape_Leng', 'Shape_Area'], inplace=True)
sub_gdf

,AREA_NAME,NSO_CODE,geometry
0,DAGORETTI,4701,"POLYGON ((36.72163 -1.26601, 36.72221 -1.26648..."
1,EMBAKASI,4702,"POLYGON ((36.91617 -1.26113, 36.91714 -1.26273..."
2,KAMUKUNJI,4703,"POLYGON ((36.87152 -1.26163, 36.87268 -1.26182..."
3,KASARANI,4704,"POLYGON ((36.91943 -1.16077, 36.92025 -1.16087..."
4,KIBRA,4705,"POLYGON ((36.82240 -1.29543, 36.82263 -1.29588..."
5,LANG'ATA,4706,"POLYGON ((36.68822 -1.29808, 36.68891 -1.29829..."
6,MAKADARA,4707,"POLYGON ((36.88239 -1.27471, 36.88259 -1.27560..."
7,MATHARE,4708,"POLYGON ((36.87761 -1.24912, 36.87775 -1.24939..."
8,NJIRU,4709,"POLYGON ((37.08141 -1.23501, 37.08931 -1.23110..."
9,STAREHE,4710,"POLYGON ((36.83953 -1.26339, 36.84010 -1.26341..."


In [121]:
#rename columns
health_sub_pop.rename(columns={'name':'AREA_NAME'}, inplace=True)
health_sub_pop

,AREA_NAME,Male,Female,Intersex,Total,Land Area (Sq. Km),Population Density (No. per Sq. Km),Health_Facilities
1,EMBAKASI,492476,496270,62.0,988808,86.3,11460,145
3,KASARANI,381234,399385,37.0,780656,86.2,9058,67
0,DAGORETTI,217651,216526,31.0,434208,29.1,14908,105
9,WESTLANDS,153818,155021,15.0,308854,97.5,3167,71
2,KAMUKUNJI,136670,131599,7.0,268276,10.5,25455,59
8,STAREHE,109173,101238,12.0,210423,20.6,10205,137
7,MATHARE,106522,100028,14.0,206564,3.0,68941,16
5,LANG'ATA,96698,100774,17.0,197489,216.8,911,68
6,MAKADARA,96369,93157,10.0,189536,11.7,16150,51
4,KIBRA,94199,91569,9.0,185777,12.1,15311,81


In [122]:
# merging the two datasets
sub_gdf_pop = health_sub_pop.merge(sub_gdf, on='AREA_NAME', how='right')

#convert to geodatframe
sub_gdf_pop = gpd.GeoDataFrame(sub_gdf_pop)

sub_gdf_pop

,AREA_NAME,Male,Female,Intersex,Total,Land Area (Sq. Km),Population Density (No. per Sq. Km),Health_Facilities,NSO_CODE,geometry
0,DAGORETTI,217651.0,216526.0,31.0,434208.0,29.1,14908.0,105.0,4701,"POLYGON ((36.72163 -1.26601, 36.72221 -1.26648..."
1,EMBAKASI,492476.0,496270.0,62.0,988808.0,86.3,11460.0,145.0,4702,"POLYGON ((36.91617 -1.26113, 36.91714 -1.26273..."
2,KAMUKUNJI,136670.0,131599.0,7.0,268276.0,10.5,25455.0,59.0,4703,"POLYGON ((36.87152 -1.26163, 36.87268 -1.26182..."
3,KASARANI,381234.0,399385.0,37.0,780656.0,86.2,9058.0,67.0,4704,"POLYGON ((36.91943 -1.16077, 36.92025 -1.16087..."
4,KIBRA,94199.0,91569.0,9.0,185777.0,12.1,15311.0,81.0,4705,"POLYGON ((36.82240 -1.29543, 36.82263 -1.29588..."
5,LANG'ATA,96698.0,100774.0,17.0,197489.0,216.8,911.0,68.0,4706,"POLYGON ((36.68822 -1.29808, 36.68891 -1.29829..."
6,MAKADARA,96369.0,93157.0,10.0,189536.0,11.7,16150.0,51.0,4707,"POLYGON ((36.88239 -1.27471, 36.88259 -1.27560..."
7,MATHARE,106522.0,100028.0,14.0,206564.0,3.0,68941.0,16.0,4708,"POLYGON ((36.87761 -1.24912, 36.87775 -1.24939..."
8,NJIRU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4709,"POLYGON ((37.08141 -1.23501, 37.08931 -1.23110..."
9,STAREHE,109173.0,101238.0,12.0,210423.0,20.6,10205.0,137.0,4710,"POLYGON ((36.83953 -1.26339, 36.84010 -1.26341..."


In [123]:
#exporting to shapefiles
'''health_gdf.to_file(outpath/'export_health_facilities.shp')
sub_gdf_pop.to_file(outpath/'Sub_Counties_Pop.shp')'''

"health_gdf.to_file(outpath/'export_health_facilities.shp')\nsub_gdf_pop.to_file(outpath/'Sub_Counties_Pop.shp')"

## Plotting Map

In [124]:
map = health_gdf.explore(
    color = 'red'
)

sub_gdf_pop.explore(
    m=map,
)

map